In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import matplotlib as mp
# import gaia_tools as gt
import scipy
from scipy.ndimage import gaussian_filter
import astropy.units as u
from astropy.coordinates import SkyCoord

In [13]:
# reading in simulation data about the LMC cdm wake
stars = np.load('C:\Data\Praktika_data\star_particles_pos_5e8.npy') # change to local path depending on whether you're using home PC / laptop
starsdf = pd.DataFrame(stars)
starsdf = starsdf.rename(columns={0:"x", 1:"y", 2:"z"})

# to remove outliers in the z axis with np.where, I first need to convert the column to a numeric type with pd.to_numeric.
# z axis is chosen specifically because, if the data is plotted in the x-z axis, the relevant data is clearly bound between -20 and 20 kpc.
# This 

#starsdf["z"] = pd.to_numeric(starsdf["z"], errors="coerce")
#starsdf['z'] = np.where((starsdf['z'] > 20) | (starsdf['z'] < -20), np.nan, starsdf['z'])
#starsdf = starsdf.dropna(subset=["z"])

x_s = starsdf['x']
y_s = starsdf['y']
z_s = starsdf['z']

bins = (35, 35)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dnurm\AppData\Local\Temp\ipykernel_18404\4014383063.py:2: SyntaxWarning: invalid escape sequence '\D'
  stars = np.load('C:\Data\Praktika_data\star_particles_pos_5e8.npy') # change to local path depending on whether you're using home PC / laptop


In [14]:
LMC_sim = [(x_s[0], 0, 0), (x_s[67000000], 0, 0), (x_s[67108863], 0, 0), (x_s[68000000], 0, 0), (x_s[134217727], 0, 0)]

pts = np.array(LMC_sim, dtype=float)
LMC_X, LMC_Y, LMC_Z = pts[:,0], pts[:,1], pts[:,2]
LMC_sim = np.array(LMC_sim)


In [15]:
df_test = pd.DataFrame({(0, -45, -45), 
                   (0, 200, -200),
                   (0, 500, 140)}, 
#},
                  columns=['x', 'y', 'z'])

x_test = df_test['x']
y_test = df_test['y']
z_test = df_test['z']

In [16]:
#trying out translation. 
LMC_cloud = [(0, 300, 0), (0, 300, 0), (0, 300, 0), (0, 300, 0), (0, 300, 0)]

LMC_sim1 = np.array(LMC_sim) + np.array(LMC_cloud)

In [17]:
# Mock wake generator
def generate_mock_wake(
    
    N_total=300_000,
    box_half_size=300.0,                 # kpc; box is [-L, L]^3
    f_overdense=0.25,                    # fraction of particles in wake component
    mu=(-30.0, 5.0, 0.0),                # kpc; wake center (x0, y0, z0)
    sigma_major=70.0,                    # kpc; wake length-scale along major axis
    sigma_minor=50.0,                    # kpc; width in the orthogonal in-plane axis
    sigma_z=8.0,                         # kpc; thickness
    theta_deg=0.0,                       # rotation of wake in XY plane (deg)
    rng_seed=7
):
    """
    Return a structured ndarray with fields: x,y,z,component (1=wake, 0=bg).
    """
    rng = np.random.default_rng(rng_seed)
    N_over = int(N_total * f_overdense)
    N_bg   = N_total - N_over
    mu = np.asarray(mu, dtype=float)

    # Rotation in the XY plane
    theta = np.deg2rad(theta_deg)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array([[c, -s],
                  [s,  c]])

    # Covariance in XY for the anisotropic Gaussian
    Lambda_xy = np.diag([sigma_major**2, sigma_minor**2])
    Sigma_xy  = R @ Lambda_xy @ R.T

    # Wake samples
    xy_over = rng.multivariate_normal(mean=mu[:2], cov=Sigma_xy, size=N_over)
    z_over  = rng.normal(loc=mu[2], scale=sigma_z, size=N_over)

    # Clip to keep inside box
    xy_over[:, 0] = np.clip(xy_over[:, 0], -box_half_size, box_half_size)
    xy_over[:, 1] = np.clip(xy_over[:, 1], -box_half_size, box_half_size)
    z_over        = np.clip(z_over,        -box_half_size, box_half_size)

    # Uniform background
    x_bg = rng.uniform(-box_half_size, box_half_size, size=N_bg)
    y_bg = rng.uniform(-box_half_size, box_half_size, size=N_bg)
    z_bg = rng.uniform(-box_half_size, box_half_size, size=N_bg)

    # Concatenate positions
    x = np.concatenate([xy_over[:, 0], x_bg])
    y = np.concatenate([xy_over[:, 1], y_bg])
    z = np.concatenate([z_over,        z_bg])

    # Pack catalog
    comp = np.concatenate([np.ones(N_over, dtype=np.int8),
                           np.zeros(N_bg, dtype=np.int8)])  # 1=wake, 0=bg
    

    cat = np.zeros(N_total, dtype=[('x','f4'),('y','f4'),('z','f4'),('component','i1')])
    
    cat['x'], cat['y'], cat['z']   = x, y, z
    cat['component'] = comp
    return cat


In [18]:
result = generate_mock_wake()
result

array([( -29.913889,   19.937277,    1.949072 , 1),
       ( -49.18965 ,  -39.52959 ,   -2.6321273, 1),
       ( -61.826954,  -44.58233 ,   -4.129546 , 1), ...,
       ( -67.989845,  105.101776, -265.415    , 0),
       (-194.56839 , -122.20985 , -250.83775  , 0),
       ( 215.33652 ,  -90.41174 , -102.19617  , 0)],
      shape=(300000,), dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('component', 'i1')])

In [19]:
df = pd.DataFrame(result)
df
coords = df[['x', 'y', 'z']].values

In [20]:
# 3D rotation matrixes
def rx(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0, 0],
                     [0, c,-s],
                     [0, s, c]])
def ry(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[ c, 0, s],
                     [ 0, 1, 0],
                     [-s, 0, c]])
def rz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,-s, 0],
                     [s, c, 0],
                     [0, 0, 1]])

def rotate(rotations, matrix):
    return rotations @ matrix


In [21]:
# testing whether rotation matrices work
tiben = rotate([300, 0, 0], rz((np.pi/2)))  
tiben 

array([ 1.8369702e-14, -3.0000000e+02,  0.0000000e+00])

In [22]:
tiben2 = rotate(ry(np.pi/2), [300, 0, 0])
tiben2

array([ 1.8369702e-14,  0.0000000e+00, -3.0000000e+02])

Exercise: test rotation function with 2 points around z axis to check how it works with more than 1 points. Check how python rotations more than one point basically. 

Test how numpy works while applying a scalar mult. to a set of vectors?

Look into: https://numpy.org/doc/stable/user/basics.broadcasting.html

In [23]:
# Current task is to try and rotate the entire set of points around a certain axis.